# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [4]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('/Users/laurabarbosa/Desktop/production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [5]:
from sklearn.model_selection import train_test_split

# Separate features (X) and target (Y)
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

# Split the data into training and testing sets (70-30 split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The `random_state` in scikit-learn’s `train_test_split` function sets the seed for random number generation. By fixing `random_state=42`, or any integer, we control the data split. This makes it possible to get the same train-test split each time the code is run. 

### Why is this useful? 

Reproducibility. It ensures that others can replicate the same results with the same data and parameters. In data science, reproducibility is crucial—it lets findings be verified and trusted.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Define numerical and categorical columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = [
    'workclass', 'education', 'marital-status', 'occupation', 
    'relationship', 'race', 'sex', 'native-country'
]

# Define transformations for numerical features
num_transforms = Pipeline(steps=[
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('robust_scaler', RobustScaler())
])

# Define transformations for categorical features
cat_transforms = Pipeline(steps=[
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Create column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transforms, numerical_features),
        ('cat_transforms', cat_transforms, categorical_features)
    ]
)



## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn import set_config

# Create the model pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Set display configuration to show pipeline structure
set_config(display='diagram')
pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robust_scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [19]:
from sklearn.model_selection import cross_validate

# Define the scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation with 5 folds
cv_results = cross_validate(
    pipeline, X_train, Y_train, 
    cv=5, 
    scoring=scoring,
    return_train_score=True
)

# Display the training and validation results
for metric in scoring.keys():
    print(f"{metric.capitalize()} - Train: {cv_results[f'train_{metric}'].mean():.4f} ± {cv_results[f'train_{metric}'].std():.4f}")
    print(f"{metric.capitalize()} - Validation: {cv_results[f'test_{metric}'].mean():.4f} ± {cv_results[f'test_{metric}'].std():.4f}\n")


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Neg_log_loss - Train: -0.0814 ± 0.0004
Neg_log_loss - Validation: -0.3659 ± 0.0241

Roc_auc - Train: 1.0000 ± 0.0000
Roc_auc - Validation: 0.9037 ± 0.0019

Accuracy - Train: 1.0000 ± 0.0000
Accuracy - Validation: 0.8521 ± 0.0058

Balanced_accuracy - Train: 1.0000 ± 0.0001
Balanced_accuracy - Validation: 0.7734 ± 0.0061



In [26]:
cv_results_df = pd.DataFrame(cv_results)
cv_results_df

,fit_time,score_time,test_accuracy,train_accuracy,test_f1,train_f1,test_precision,train_precision,test_recall,train_recall,test_roc_auc,train_roc_auc,test_neg_log_loss,train_neg_log_loss,test_neg_brier_score,train_neg_brier_score
0,6.192281,0.193553,0.852602,1.000000,0.666435,1.000000,0.732620,1.000000,0.611217,1.000000,0.901837,1.0,-0.368913,-0.080076,-0.103222,-0.014218
1,6.167419,0.187671,0.849201,0.999962,0.659028,0.999920,0.723323,0.999841,0.605230,1.000000,0.895136,1.0,-0.412081,-0.079690,-0.106368,-0.014062
2,5.994561,0.204045,0.851351,0.999885,0.674074,0.999761,0.713980,0.999841,0.638393,0.999681,0.902903,1.0,-0.363730,-0.080478,-0.103267,-0.014392
3,6.070991,0.201556,0.860565,1.000000,0.684064,1.000000,0.752680,1.000000,0.626913,1.000000,0.901963,1.0,-0.382151,-0.081122,-0.102357,-0.014385
4,6.121136,0.182825,0.857187,0.999962,0.680851,0.999920,0.736999,1.000000,0.632653,0.999841,0.906197,1.0,-0.366627,-0.081289,-0.101029,-0.014389


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [31]:
import pandas as pd
from sklearn.model_selection import cross_validate

# Assuming 'pipeline', 'X', and 'Y' are already defined
scoring = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc', 'neg_log_loss', 'neg_brier_score']
cv_results = cross_validate(pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)

# Convert results to DataFrame and sort by negative log loss of the test set
cv_results_df = pd.DataFrame(cv_results)
sorted_cv_results_df = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)

# Display the sorted DataFrame
print(sorted_cv_results_df)


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


   fit_time  score_time  test_accuracy  train_accuracy   test_f1  train_f1  \
2  6.005345    0.183964       0.851351        0.999885  0.670972  0.999761   
0  6.231214    0.196710       0.856287        0.999962  0.675226  0.999920   
3  6.353995    0.209320       0.859029        1.000000  0.679916  1.000000   
4  6.235733    0.231339       0.854730        0.999962  0.675805  0.999920   
1  5.986951    0.186604       0.847973        0.999923  0.656250  0.999841   

   test_precision  train_precision  test_recall  train_recall  test_roc_auc  \
2        0.718341         0.999681     0.629464      0.999841      0.903863   
0        0.741051         0.999841     0.620140      1.000000      0.902195   
3        0.750000         1.000000     0.621811      1.000000      0.902632   
4        0.730370         1.000000     0.628827      0.999841      0.905833   
1        0.720274         1.000000     0.602679      0.999681      0.897290   

   train_roc_auc  test_neg_log_loss  train_neg_log_loss 

Calculate the mean of each metric. 

In [41]:
mean_metrics = cv_results_df.mean()
print(mean_metrics)


fit_time                 6.162648
score_time               0.201587
test_accuracy            0.853874
train_accuracy           0.999946
test_f1                  0.671634
train_f1                 0.999888
test_precision           0.732007
train_precision          0.999904
test_recall              0.620584
train_recall             0.999872
test_roc_auc             0.902362
train_roc_auc            1.000000
test_neg_log_loss       -0.371583
train_neg_log_loss      -0.080501
test_neg_brier_score    -0.102943
train_neg_brier_score   -0.014305
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [42]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

# Generate predictions and prediction probabilities on the test set
Y_pred = pipeline.predict(X_test)
Y_pred_proba = pipeline.predict_proba(X_test)

# Calculate performance metrics
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),  # Negative log loss
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),  # ROC AUC for the positive class
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

# Display the results
print("Test Performance Metrics:")
print(test_metrics)


Test Performance Metrics:
{'neg_log_loss': -0.38375581413875526, 'roc_auc': 0.9003514895190794, 'accuracy': 0.8531067663015662, 'balanced_accuracy': 0.7730767491726505}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recoding `income` as a binary variable (0 or 1) right at the start makes everything smoother. It aligns with how `scikit-learn` models and metrics work, especially for binary classification. This way, we don’t have to repeatedly convert it, saving memory and keeping everything consistent and efficient throughout the process.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.